In [ ]:
!pip install scikit-multiflow

In [3]:
import numpy as np

np.float = np
np.int = np

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from skmultiflow.data import SEAGenerator
from skmultiflow.meta import AccuracyWeightedEnsembleClassifier as AWE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

window_size = 1000
max_iterations = 10

stream = SEAGenerator()

pre_train_size = 100
X_pre_train, y_pre_train = stream.next_sample(pre_train_size)

model = AWE()
model.fit(X_pre_train, y_pre_train)

accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

# Janela deslizante com avaliação prequential
for i in range(max_iterations):
    X_batch, y_batch = stream.next_sample(window_size)  # new batch
    model.fit(X_batch, y_batch)

    X_validation, y_validation = stream.next_sample(window_size)
    y_pred = model.predict(X_validation)

    accuracy = accuracy_score(y_validation, y_pred)
    precision = precision_score(y_validation, y_pred, average='weighted')
    recall = recall_score(y_validation, y_pred, average='weighted')
    f1 = f1_score(y_validation, y_pred, average='weighted')

    accuracy_list.append(accuracy if not np.isnan(accuracy) else 0)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

    print(f"Iteração {i+1}/{max_iterations}")
    print(f"Métricas acumuladas até agora: {accuracy_list}")

# Calcular as médias das métricas
mean_accuracy = np.mean(accuracy_list)
mean_precision = np.mean(precision_list)
mean_recall = np.mean(recall_list)
mean_f1 = np.mean(f1_list)

# Exibir os resultados
print("Médias das Métricas de Avaliação:")
print(f"Acurácia: {mean_accuracy}")
print(f"Precisão: {mean_precision}")
print(f"Recall: {mean_recall}")
print(f"F1-Score: {mean_f1}")
